In [4]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

# Display all of the columns
pd.set_option('display.max_columns', None)

In [18]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path('data/board_games_complete.csv')
)
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [10]:
df["category"] = df["category"].str.split(",")

In [25]:
# Process data
df['category_clean'] = df['category_clean'].replace({'Animals': 'Other','Card Game': 'Other', 'Dice': 'Other','Electronic': 'Other','Game System': 'Other','Horror': 'Other','Humor': 'Other','Mature / Adult': 'Other','Miniatures': 'Other','Music': 'Other','none': 'Other','Party Game': 'Other','Religious': 'Other','Fighting': 'Other','Racing': 'Other','Sports': 'Other','Age of Reason': 'Era','American West': 'Era','Ancient': 'Era','Medieval': 'Era','Napoleonic': 'Era','Post-Napoleonic': 'Era','Prehistoric': 'Era','Renaissance': 'Era','Abstract Strategy': 'Strategy','Bluffing': 'Strategy','City Building': 'Strategy','Civilization': 'Strategy','Collectible Components': 'Strategy','Deduction': 'Strategy','Economic': 'Strategy','Exploration': 'Strategy','Farming': 'Strategy','Industry / Manufacturing': 'Strategy','Mafia': 'Strategy','Maze': 'Strategy','Murder/Mystery': 'Strategy','Negotiation': 'Strategy','Pirates': 'Strategy','Political': 'Strategy','Puzzle': 'Strategy','Real-time': 'Strategy','Spies/Secret Agents': 'Strategy','Territory Building': 'Strategy','Trains': 'Strategy','Transportation': 'Strategy','Zombies': 'Strategy','Action / Dexterity': 'Adventure','Adventure': 'Adventure','Arabian': 'Adventure','Aviation / Flight': 'Adventure','Comic Book / Strip': 'Adventure','Fantasy': 'Adventure','Movies / TV / Radio theme': 'Adventure','Mythology': 'Adventure','Nautical': 'Adventure','Pike and Shot': 'Adventure','Print & Play': 'Adventure','Science Fiction': 'Adventure','Space Exploration': 'Adventure','Travel': 'Adventure','Video Game Theme': 'Adventure','Book': 'Educational',"Children's Game": 'Educational','Educational': 'Educational','Environmental': 'Educational','Math': 'Educational','Medical': 'Educational','Memory': 'Educational','Novel-based': 'Educational','Number': 'Educational','Trivia': 'Educational','Word Game': 'Educational','American Civil War': 'Wargame','American Indian Wars': 'Wargame','American Revolutionary War': 'Wargame','Civil War': 'Wargame','Korean War': 'Wargame','Modern Warfare': 'Wargame','Vietnam War': 'Wargame','Wargame':'Wargame','World War I':'Wargame','World War II':'Wargame',})

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  5000 non-null   int64  
 1   bgg_url               5000 non-null   object 
 2   id                    5000 non-null   int64  
 3   name                  5000 non-null   object 
 4   minplayers            5000 non-null   int64  
 5   maxplayers            5000 non-null   int64  
 6   playingtime           5000 non-null   int64  
 7   minplaytime           5000 non-null   int64  
 8   maxplaytime           5000 non-null   int64  
 9   yearpublished         5000 non-null   int64  
 10  average_rating        5000 non-null   float64
 11  geek_rating           5000 non-null   float64
 12  users_rated           5000 non-null   int64  
 13  image_url             5000 non-null   object 
 14  thumb_url             5000 non-null   object 
 15  age                  

In [26]:
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Other, Other",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Educational,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Other, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Other,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Strategy,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Wargame,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [23]:
df1 = df
words_to_replace = ['Animals', 'Dice', 'Card Game', 'Electronic','Game System','Horror','Humor','Mature / Adult','Miniatures','Music','none','Party Game','Religious','Fighting','Racing','Sports',]
replacement_word = 'Other'
for word in words_to_replace:
    df1['category'] = df1['category'].str.replace(word, replacement_word)

In [ ]:
df1 = df
words_to_replace = []
replacement_word = 'Era'
for word in words_to_replace:
    df1['category'] = df1['category'].str.replace(word, replacement_word)

In [24]:
df1.head(50)

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Other, Other",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Other, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785
5,6,https://boardgamegeek.com/boardgame/187645/sta...,187645,Star Wars: Rebellion,2,4,240,180,240,2016,8.45420,8.16391,15740,https://cf.geekdo-images.com/original/img/unCB...,https://cf.geekdo-images.com/thumb/img/OF1Hi31...,14,"Area Control / Area Influence, Area Movement, ...",23989,"Other, Other, Movies / TV / Radio theme, Scien...",Corey Konieczka,Fantasy Flight Games,3.6698,NaN,NaN,1594,6,Area Control / Area Influence,5,Fighting,1,Corey Konieczka,Star Wars Rebellion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,https://boardgamegeek.com/boardgame/169786/scythe,169786,Scythe,1,5,115,90,115,2016,8.29130,8.12066,36728,https://cf.geekdo-images.com/original/img/enxC...,https://cf.geekdo-images.com/thumb/img/ZpuWhZu...,14,"Area Control / Area Influence, Grid Movement, ...",47847,"Economic, Other, Science Fiction, Territory Bu...",Jamey Stegmaier,Stonemaier Games,3.3694,NaN,NaN,3968,3,Area Control / Area Influence,4,Economic,1,Jamey Stegmaier,Scythe,bo

In [ ]:
# Replace these with your data and preferences 
data = np.array([[category, era, wargame, strategy, adventure, educational, other], 
                [mechanic, role_playing, acting],
                [complexity, low, moderate, expert], 
                [age, everyone, child, adult],
                [minplayers, feature1_2, feature2_2, ...],
                [maxplayers, feature1_2, feature2_2, ...],
                [playingtime, short, medium, long],
                 ... ]) 
# Your data matrix where each row represents an item with its features 


In [ ]:
user_preferences = np.array([user_preference_category, user_preference_mechanic, user_preference_complexity, user_preference_age, user_preference_minplayers, user_preference_maxplayers, user_preference_playingtime]) 
# User's preferences or ratings

In [ ]:
# Number of neighbors to consider (you can change this) 
k_neighbors = 5 

In [ ]:
# Initialize the k-NN model 
knn = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean') 
# You can choose a different distance metric 

In [ ]:
# Fit the model to your data 
knn.fit(data[:, 1:]) 
# Exclude the item IDs when fitting the model 

In [ ]:
# Find the k nearest neighbors for user's preferences distances, 
indices = knn.kneighbors([user_preferences])

In [ ]:
# Recommend items based on the nearest neighbors 
recommended_items = data[indices[0], 0] 
# Get the item IDs of the recommended items 


In [ ]:
# You can print or return the recommended items 
print("Recommended Items:", recommended_items) 